In [9]:
import pandas as pd 
import numpy as np
input_df = pd.read_csv('../data/train/input_2023_w01.csv')
output_df = pd.read_csv('../data/train/output_2023_w01.csv')
supplemental_df = pd.read_csv('../data/supplementary_data.csv')
pd.set_option('display.max_columns', None)

df = pd.merge(input_df, supplemental_df, on=['game_id','play_id'], how='left')
print(df.head())




/var/folders/7d/j6bnzr6j1s9dc0n2tggc7syr0000gp/T/ipykernel_99767/1444866542.py:5: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  supplemental_df = pd.read_csv('../data/supplementary_data.csv')


      game_id  play_id  player_to_predict  nfl_id  frame_id play_direction  \
0  2023090700      101              False   54527         1          right   
1  2023090700      101              False   54527         2          right   
2  2023090700      101              False   54527         3          right   
3  2023090700      101              False   54527         4          right   
4  2023090700      101              False   54527         5          right   

   absolute_yardline_number player_name player_height  player_weight  \
0                        42  Bryan Cook           6-1            210   
1                        42  Bryan Cook           6-1            210   
2                        42  Bryan Cook           6-1            210   
3                        42  Bryan Cook           6-1            210   
4                        42  Bryan Cook           6-1            210   

  player_birth_date player_position player_side         player_role      x  \
0        1999-09-07 

In [8]:

# Filtering out Zone coverages
df_man =  df[~df['team_coverage_type'].str.contains("ZONE")]
print(df_man['team_coverage_type'].value_counts())


COVER_1_MAN    60120
COVER_0_MAN     7546
COVER_2_MAN     5605
Name: team_coverage_type, dtype: int64


In [11]:
#hello

df_man['x_player_ball_dist'] = df_man['ball_land_x']-df_man['x']
df_man['y_player_ball_dist'] = df_man['ball_land_y']-df_man['y']
print(df_man.head())

        game_id  play_id  player_to_predict  nfl_id  frame_id play_direction  \
234  2023090700      194               True   44888         1           left   
235  2023090700      194               True   44888         2           left   
236  2023090700      194               True   44888         3           left   
237  2023090700      194               True   44888         4           left   
238  2023090700      194               True   44888         5           left   

     absolute_yardline_number    player_name player_height  player_weight  \
234                        89  Alex Anzalone           6-3            240   
235                        89  Alex Anzalone           6-3            240   
236                        89  Alex Anzalone           6-3            240   
237                        89  Alex Anzalone           6-3            240   
238                        89  Alex Anzalone           6-3            240   

    player_birth_date player_position player_side       